In [1]:
import numpy as np
import pandas as pd 
import random 
import copy 
import warnings
import sys
import pickle
import pyreadr as py
import itertools

from tqdm import tqdm
from copy import deepcopy
from scipy.io import loadmat
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

sys.path.append('../../')
from ACME.ACME import ACME
from ACME.visual_utils import * 
sys.path.remove('../../')

warnings.filterwarnings('ignore')

# set seed for reproducibility
np.random.seed(0)
random.seed(0)

# AcME-AD to explain IF in TEP dataset

We subsample the original dataset to resort to a typical anomaly detection scenario where anomalies are rare. 

In [2]:
n_normal_simulations = 70 
n_faulty_simulations = 3

In [3]:
# # load the dataset and sample n_normal_simulations and n_faulty_simulations
# normal_data = py.read_r('ad_industrial_datasets/TEP_FaultFree_Training.RData')['fault_free_training']
# normal_data = normal_data[normal_data['simulationRun'].isin(random.sample(list(normal_data['simulationRun'].unique()), n_normal_simulations))].reset_index(drop=True)
# normal_data['Target'] = 0

# fault_data = py.read_r('ad_industrial_datasets/TEP_Faulty_Training.RData')['faulty_training']
# fault_data = fault_data[fault_data['simulationRun'].isin(random.sample(list(fault_data['simulationRun'].unique()), n_faulty_simulations))].reset_index(drop=True)
# fault_data['Target'] = 1

# # remove the first 20 samples from fault_data because are not faulty 
# fault_data = fault_data.groupby('simulationRun').apply(lambda x: x.iloc[20:]).reset_index(drop=True)

# # save this data for future use
# normal_data.to_csv('ad_industrial_datasets/TEP_FaultFree_Training_subsample_70_3.csv', index=False)
# fault_data.to_csv('ad_industrial_datasets/TEP_Faulty_Training_subsample_70_3_removedfirst20.csv', index=False)

In [4]:
# load the data
normal_data = pd.read_csv('ad_industrial_datasets/TEP_FaultFree_Training_subsample_70_3.csv')
fault_data = pd.read_csv('ad_industrial_datasets/TEP_Faulty_Training_subsample_70_3_removedfirst20.csv')

In [5]:
# create 20 datasets, each one containing only 1 specific fault 
faulty_data, data = [], []
for i in range(20): 
    faulty_data.append(fault_data[fault_data['faultNumber'] == i+1].reset_index(drop=True))
    data.append(pd.concat([normal_data, faulty_data[i]]).reset_index(drop=True))

In [6]:
features = data[0].columns[3:-1] 
contamination = faulty_data[0].shape[0] / data[0].shape[0]

## Training
Train Isolation Forest on each dataset and evaluate it. 

In [7]:
results = []
for i in range(20): 
    ad_model = IsolationForest(n_estimators=200, max_samples=128, contamination=contamination, random_state=0, n_jobs=-1).fit(data[i][features])
    data[i]['Prediction'] = ad_model.predict(data[i][features])
    data[i]['Prediction'] = data[i]['Prediction'].apply(lambda x: 1 if x == -1 else 0) # need to convert the predictions to 0 for normal and 1 for faulty

    prec, rec, f1, _ = precision_recall_fscore_support(data[i]['Target'], data[i]['Prediction'], average='binary')
    avg_prec = average_precision_score(data[i]['Target'], data[i]['Prediction'])

    results.append({'Fault': i+1, 'Precision': prec, 'Recall': rec, 'F1': f1, 'Average Precision': avg_prec})

results = pd.DataFrame(results)
display(results)

,Fault,Precision,Recall,F1,Average Precision
0,1,0.939583,0.939583,0.939583,0.885204
1,2,0.934858,0.899333,0.916752,0.844886
2,3,0.072072,0.069333,0.070676,0.043244
3,4,0.146916,0.141333,0.144071,0.056052
4,5,0.391545,0.376667,0.383962,0.173099
5,6,0.973666,0.936667,0.954808,0.914603
6,7,0.635482,0.611333,0.623174,0.404464
7,8,0.918226,0.883333,0.900442,0.815894
8,9,0.072072,0.069333,0.070676,0.043244
9,10,0.228690,0.220000,0.224261,0.082367


## Explain fault 12
Performances are good on fault 12 and we have prior knowledge on the fact that xmeas 11 is 'root cause' of the fault. 
[Harinarayan, R. Rajesh Alias, and S. Mercy Shalinie. "XFDDC: eXplainable Fault Detection Diagnosis and Correction framework for chemical process systems." Process Safety and Environmental Protection 165 (2022): 463-474.]

In [8]:
data_12 = data[11].copy()

In [9]:
# hyperparameters tuning 
from sklearn.model_selection import ParameterGrid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_samples': [64, 128, 256]
}
best_avg_prec = 0
best_config = None

for config in ParameterGrid(param_grid):
    iforest = IsolationForest(contamination=contamination, random_state=0, n_jobs=-1, **config).fit(data_12[features])
    data_12['Prediction'] = iforest.predict(data_12[features])
    data_12['Prediction'] = data_12['Prediction'].apply(lambda x: 1 if x == -1 else 0)

    avg_prec = average_precision_score(data_12['Target'], data_12['Prediction'])
    if avg_prec > best_avg_prec: 
        best_avg_prec = avg_prec
        best_config = config

print("Best config: ", best_config, " | Best avg prec: ", best_avg_prec)

Best config:  {'max_samples': 256, 'n_estimators': 200}  | Best avg prec:  0.794499498444704


In [10]:
# define score function needed for AcME-AD
def if_score_function(model, X): 
    return 0.5 * (- model.decision_function(X) + 1)

# train isolation forest with the best config
ad_model = IsolationForest(contamination=contamination, random_state=0, n_jobs=-1, **best_config).fit(data_12[features])
data_12['Prediction'] = ad_model.predict(data_12[features])
data_12['Prediction'] = data_12['Prediction'].apply(lambda x: 1 if x == -1 else 0)
data_12['Score'] = if_score_function(ad_model, data_12[features])

In [11]:
# Confusion matrix 
n_normal_as_faulty = data_12[(data_12['Target'] == 0) & (data_12['Prediction'] == 1)].shape[0]
n_faulty_as_faulty = data_12[(data_12['Target'] == 1) & (data_12['Prediction'] == 1)].shape[0]
n_normal_as_normal = data_12[(data_12['Target'] == 0) & (data_12['Prediction'] == 0)].shape[0]
n_faulty_as_normal = data_12[(data_12['Target'] == 1) & (data_12['Prediction'] == 0)].shape[0]

print("Normal as faulty: ", n_normal_as_faulty, " | Faulty as faulty: ", n_faulty_as_faulty, " | Normal as normal: ", n_normal_as_normal, " | Faulty as normal: ", n_faulty_as_normal)

conf_matrix = np.array([[n_normal_as_normal, n_faulty_as_normal], [n_normal_as_faulty, n_faulty_as_faulty]])
print(conf_matrix)

Normal as faulty:  136  | Faulty as faulty:  1307  | Normal as normal:  34864  | Faulty as normal:  193
[[34864   193]
 [  136  1307]]


## AcME-AD explanations

In [12]:
data_to_explain = data_12[data_12['Prediction'] == 1].reset_index(drop=True)

In [13]:
# # uncomment this part to generate your own explanations
# acme = ACME(ad_model, 'Score', features, task='ad', score_function=if_score_function)
# acme = acme.explain(data_12, robust = True)

# local_explanations = []
# rankings = pd.DataFrame(columns = features)
# for j in tqdm(data_to_explain.index): 
#     local_exp = acme.explain_local(data_to_explain.loc[j])
#     feature_table = local_exp.feature_importance(local=True, weights={'delta':0.3, 'change':0.3, 'distance':0.2, 'ratio':0.2})
#     ranking = feature_table['importance'].rank(ascending=False, method='min')

#     local_explanations.append(local_exp)
#     rankings.loc[j] = ranking

In [14]:
# # save data
# with open('results/TEP_IF_acme_local_exp.pkl', 'wb') as f: 
#     pickle.dump(local_explanations, f)

# rankings.to_csv('results/TEP_IF_rankings.csv', index=False)

In [15]:
# load our computed explanations 
with open('results/TEP_IF_acme_local_exp.pkl', 'rb') as f: 
    local_explanations = pickle.load(f)

rankings = pd.read_csv('results/TEP_IF_rankings.csv')

In [16]:
acme_rank_counting = rankings.apply(lambda x: x.value_counts()).fillna(0).astype(int)
acme_rank_counting = acme_rank_counting / acme_rank_counting.sum()

In [17]:
# define custom color and pattern for the features for the paper
visual_dict = {
    'xmeas_4': {"color": "limegreen", "pattern": ""}, 
    'xmeas_7': {'color': 'lightskyblue', 'pattern': '.'},  
    'xmeas_11':{"color": "midnightblue", "pattern": "x"},
    'xmeas_13':{"color": "crimson", "pattern": "/"},
    'xmeas_38':{"color":"mediumpurple" , "pattern": "\\"}, 
    'xmv_2': {"color": "aquamarine", "pattern": "+"}, 
}

acme_stacked_barplot_fig = feature_importance_distribution_barplot(acme_rank_counting, n_positions=5, threshold=0.05, color_pattern_dict=visual_dict)
acme_stacked_barplot_fig.show()

## Local explanation
Explain  random anomaly

In [18]:
random_loc_exp = random.choice(local_explanations)
what_if = random_loc_exp.summary_plot(local=True, n_features=10) 
what_if.show()
what_if.update_layout(width=100*15, height=100*6, font=dict(size=18))
what_if.write_image('results/TEP_local_random.pdf')

Using default weights for anomaly detection feature importance


Explain an anomaly with 'middle' anomaly score

In [19]:
data_to_explain['Abs_Diff'] = np.abs(data_to_explain['Score'] - 0.506)
random_anomaly = data_to_explain.loc[data_to_explain['Abs_Diff'].idxmin()]

acme_exp_2 = ACME(ad_model, 'Score', features=features, task='ad', score_function=if_score_function)
acme_exp_2.explain(data_12, robust=True)

acme_loc_example = acme_exp_2.explain_local(pd.Series(random_anomaly))
what_if_fig = acme_loc_example.summary_plot(local=True, n_features=10)
what_if_fig.show()

Using default weights for anomaly detection feature importance
